In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path

In [8]:
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    data_dir: Path
    status_file: Path
    all_schema: dict

In [5]:
from src.Customer_segementation.utils.common import read_yaml, create_directories
from src.Customer_segementation.constant import *

In [13]:
class configurationManager: 
    def __init__(self,config_file_path=CONFIG_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.schema=read_yaml(schema_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_Data_validation_config(self)->DataValidationConfig:
        config=self.config.data_validation
        schema=self.schema.COLUMNS
        create_directories([config.root_dir])

        data_validation_config=DataValidationConfig(
            root_dir=config.root_dir,
            data_dir=config.data_dir,
            status_file=config.status_file,
            all_schema=schema
            )
        return data_validation_config
        

In [14]:
from src.Customer_segementation.logger import logger
import pandas as pd

In [29]:
class DataValidation:
    def __init__(self, config=DataValidationConfig):
        self.config=config

    def validation_all_columns(self):
        """
        Method_name: validation all  columns
        Description: This method validate all columns like data comming from database and columns use for model.

        output: function return txt file, it columns all right than return True otherwise false.
        on failure: write exception log and raise exception.
        """
        logger.info(f"Data validation started in data validation class")
        try: 
            validation_satus=None
            data=pd.read_csv(self.config.data_dir)
            logger.info(f"Customer segemntaion data: {data.head()}")

            all_columns=list(data.columns)
            logger.info(f"All Columns of data {all_columns}")

            logger.info(f"All schema data: {self.config.all_schema}")

            schema=list(self.config.all_schema.keys())

            logger.info(f"Actual schema : {schema}")

            for col in all_columns: 
                if col not in schema:
                    validation_satus=False
                    with open(self.config.status_file,'w') as f: 
                        f.write(f"Validation status: {validation_satus}")

                else: 
                    validation_satus=True
                with open(self.config.status_file,'w') as f: 
                    f.write(f"Validation status: {validation_satus}")

        except Exception as e: 
            raise e

In [30]:
try: 
    configmanger=configurationManager()
    data_validationConfig=configmanger.get_Data_validation_config()
    data_validation=DataValidation(data_validationConfig)
    data_validation.validation_all_columns()
except Exception as e: 
    raise e

[2024-06-25 16:44:50,574 : INFO : common : Yaml file read config/config.yaml successfully]
[2024-06-25 16:44:50,577 : INFO : common : Yaml file read schema.yaml successfully]
[2024-06-25 16:44:50,578 : INFO : common : Yaml file read params.yaml successfully]
[2024-06-25 16:44:50,578 : INFO : common : Directories created ['artifacts']]
[2024-06-25 16:44:50,579 : INFO : common : Directories created ['artifacts/data_validation']]
[2024-06-25 16:44:50,579 : INFO : 3667790040 : Data validation started in data validation class]
[2024-06-25 16:44:50,593 : INFO : 3667790040 : Customer segemntaion data:        ID  Gender Ever_Married  Age Graduated     Profession  Work_Experience  \
0  462809    Male           No   22        No     Healthcare              1.0   
1  462643  Female          Yes   38       Yes       Engineer              NaN   
2  466315  Female          Yes   67       Yes       Engineer              1.0   
3  461735    Male          Yes   67       Yes         Lawyer              